<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_LaOpinion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [1]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

class LaOpinionSpider (scrapy.Spider):

    #Es obligatorio poner un nombre
    name = 'laOpinion'

    #Ponemos que el dominio que está permitido es el de la página y no vamos a irnos fuera de la misma
    allowed_domains = ['www.laopiniondemurcia.es']

    start_urls = ['https://www.laopiniondemurcia.es/deportes/2023/12/01/elpozo-murcia-busca-cabeza-serie-copa-futbol-sala-95322591.html']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    def parse (self, response):
      """
      @inherit

      @param self
      @param response
      """

      url = str(response.request.url).strip()
      #Buscamos todos los elementos en el archivo XML con la etiqueta <item>
      for item in response.css ('.bbnx-template.article-template'):

            #Obtenemos por cada elemento <item> el texto del subelemento <title>. Además co el BeautifulSoup
            #procesamos el texto en html y nos quedamos con el texto
            title = BeautifulSoup(str(item.css ('.h1::text').get()), 'html.parser').get_text().strip()
            #Obtenemos por cada elemento <item> el texto del subelemento <h2>
            subtitle = BeautifulSoup(str(item.css ('.headline-article__extended-subtitle').get()), 'html.parser').get_text().strip()

            author = BeautifulSoup(str(item.css('.news-author').get()), 'html.parser').get_text().strip()

            date = BeautifulSoup(str(item.css(".article-author__date").get()), "html.parser").get_text().strip()

            content = BeautifulSoup(str(item.css ('.article-body ').get()), 'html.parser').get_text().strip()

            #Imprimimos la información obtenida para comprobar lo que estamos extrayendo
            print ("-------------------------")
            print ('URL:' + url)
            print ("Date : "+date)
            print ('Author:' + author)
            print ('Título:' + title)
            print ('Subtitulo:' + subtitle)
            print ('Contenido:' + content)
            print ("-------------------------")

            data = {
                'url' : url,
                "date": date,
                'author': author,
                'title': title,
                'subtitle':subtitle,
                'content': content,
            }

            #Creamos para cada item un fichero json y para ello obtenemos un número aleatorio.
            filename = str(random.random()).replace(".","") + ".json"

            # Si tenemos descripción, url y título entonces lo guardamos a disco en la carpeta 'rss'
            if content and title and url:
                with open ('rss/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)




## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

# Nueva sección

In [3]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('laOpinion')):
    os.mkdir('laOpinion')

# Creamos el proceso con el RSSSpider
process.crawl(LaOpinionSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
URL:https://www.laopiniondemurcia.es/deportes/2023/12/01/elpozo-murcia-busca-cabeza-serie-copa-futbol-sala-95322591.html
Date : 01·12·23
Author:Fran Montiel
Título:ElPozo busca ser cabeza de serie en la Copa
Subtitulo:Los de Javi Rodríguez visitan hoy al Santa Coloma con el objetivo de mantenerse entre los cuatro primeros
Contenido:ElPozo Murcia afronta esta tarde su penúltimo partido de la primera vuelta ante el antiguo club de su técnico, Industrias Santa Coloma, en una nueva oportunidad para acercarse al liderato y distanciarse de sus perseguidores, a partir de las 19:30 horas.  Con la Copa de España ya asegurada bajo el brazo, los de Javi Rodríguez buscan ahora conseguir el segundo puesto y quitarse rápidamente de la cabeza el resultado del pasado fin de semana, empatando a dos ante un Ribera Navarra que venía en una dinámica muy mala situado en la parte baja de la tabla. 



        Una vez más, el problema fue la falta de ideas en tres cuarto de campo y 